In [2]:
import gradio as gr
import torch
print(gr.__version__)
print(torch.__version__)

3.23.0
1.13.1


In [3]:
from pathlib import Path

MODEL_PATH = Path("models") 
MODEL_PATH.mkdir(parents=True, exist_ok=True)

In [37]:
import json
from PIL import Image
import numpy as np
import os

with open("models/model_info.json", "r", encoding="utf-8") as f:
    models_info = json.load(f)



LANGUAGES = ['EN','CN','JP']
speaker_id = 0




In [20]:
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Path):
            return str(obj)
        return super().default(obj)


In [105]:
def add_model_fn(example_text, cover, SpeakerID, name_en, name_cn, language):



    # 检查必填字段是否为空
    if not speaker_id or not name_en or not language:
        raise gr.Error("Please fill in all required fields!")
        return "Failed to add model"




    ### 保存上传的文件

    # 生成文件路径
    model_save_dir = Path("models")
    model_save_dir = model_save_dir / name_en
    img_save_dir = model_save_dir
    model_save_dir.mkdir(parents=True, exist_ok=True)

    #shutil.copyfile(file.value,Model_save_path)

    # file_data = file.data[0]
    # filename = secure_filename(file_data.name)
    # filepath = os.path.join("models", name_en, filename)
    # os.makedirs(os.path.dirname(filepath), exist_ok=True)


    # 保存checkpoints 和 cover
    #tensor = torch.FloatTensor(file)
    Model_name = name_en + ".pth"
    model_save_dir = model_save_dir / Model_name
    #torch.save(tensor, Model_save_path)

    #
    #  # convert to RGB format if necessary
    # if len(img.shape) == 2 or img.shape[2] == 1:
    #     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGBA)
    # else:
    #     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    #
    # cv2.imwrite(str(img_save_dir / "cover.png"), img)

    if cover is not None:
        img = np.array(cover)
        img = Image.fromarray(img)
        img.save(os.path.join(img_save_dir, 'cover.png'))


    
    #获取用户输入
    new_model = {
        "name_en": name_en,
        "name_zh": name_cn,
        "cover": img_save_dir / "cover.png",
        "sid": SpeakerID,
        "example": "それに新しいお菓子屋さんも出来てみんな買いものを楽しんでいます！",
        "language": language,
        "type": "single",
        "model_path": model_save_dir
    }



    with open("models/model_info.json", "r", encoding="utf-8") as f:
        models_info = json.load(f)

    models_info[name_en] = new_model
    with open("models/model_info.json", "w") as f:
        json.dump(models_info, f, cls=CustomEncoder)








    #return file.
    return "Success"


In [102]:
def clear_add_model_info():
    return "",None,"","","",""

In [104]:
theme = gr.themes.Base()

with gr.Blocks(theme=theme) as interface:
  with gr.Tab("Settings"):
        with gr.Box():
            gr.Markdown("""# Select Model""")
            with gr.Row():

                with gr.Column(scale = 5):
                    model_choice = gr.Dropdown(label = "Model",
                                           choices=[(model["name_en"]) for name, model in models_info.items()],
                                           interactive=True,
                                           value=models_info['yuuka']['name_en']
                                         )
                with gr.Column(scale = 5):
                    speaker_id = gr.Dropdown(label = "Speaker ID",
                                         choices=[(str(model["sid"])) for name, model in models_info.items()],
                                         interactive=True,
                                         value=str(models_info['yuuka']['sid'])
                                         )

                with gr.Column(scale = 1):
                    refresh_button = gr.Button("Refresh", variant="primary")
                    reset_button = gr.Button("Reset")

        with gr.Box():
            gr.Markdown("# Add Model\n"
                        "> *为必填选项\n"
                        "> 添加完成后将**checkpoints**文件放到对应生成的文件夹中"
                        )


            with gr.Row():
                # file = gr.Files(label = "VITS Model*", file_types=[".pth"])
                example_text = gr.Textbox(label = "Example Text",
                                          lines=16,
                                          placeholder="Enter the example text here",)
                model_cover = gr.Image(label = "Cover")

                with gr.Column():
                    model_speaker_id = gr.Textbox(label = "Speaker List*",
                                                  placeholder="Single speaker model default=0")
                    model_name_en = gr.Textbox(label = "name_en*")
                    model_name_cn = gr.Textbox(label = "name_cn")
                    model_language = gr.Dropdown(label = "Language*",
                                               choices=LANGUAGES,
                                               interactive=True)
                    with gr.Row():
                        add_model_button = gr.Button("Add Model", variant="primary")
                        clear_add_model_button = gr.Button("Clear")
            with gr.Box():
              with gr.Row():
                message_box = gr.Textbox(label = "Message")



        add_model_button.click(add_model_fn,
                               inputs = [example_text, model_cover, model_speaker_id, model_name_en, model_name_cn, model_language],
                               outputs = message_box
                               )
        clear_add_model_button.click(clear_add_model_info,
                                     outputs = [example_text, model_cover, model_speaker_id, model_name_en, model_name_cn, model_language]
        )

interface.queue(concurrency_count=1).launch(debug=True)

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [62]:
import gradio as gr

def file_upload(file):
    return file.name


with gr.Blocks() as interface:

    a = gr.Files(label = "VITS Model*", file_types=[".pth"])
    b = gr.Files(label = "Cover", file_types=[".png"])
    c = gr.Button()
    d = gr.Textbox()

    c.click(fn=file_upload,inputs=a,outputs=d)


interface.queue(concurrency_count=1).launch(debug=True)

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "D:\Anaconda\envs\ML\lib\site-packages\gradio\routes.py", line 394, in run_predict
    output = await app.get_blocks().process_api(
  File "D:\Anaconda\envs\ML\lib\site-packages\gradio\blocks.py", line 1075, in process_api
    result = await self.call_function(
  File "D:\Anaconda\envs\ML\lib\site-packages\gradio\blocks.py", line 884, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "D:\Anaconda\envs\ML\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "D:\Anaconda\envs\ML\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "D:\Anaconda\envs\ML\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "C:\Users\l4227\AppData\Local\Temp\ipykernel_11412\3513495185.py", line 4, in file_upload
    return file.name
AttributeError: 'lis

Keyboard interruption in main thread... closing server.


In [66]:
import gradio as gr

def file_upload(file):
    return file.name

iface = gr.Interface(fn=file_upload, inputs="file", outputs="text")
iface.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [65]:
import gradio as gr

def get_file_path(input_file):
    # do something with input file
    return "/path/to/output/file.txt"

input_file = gr.inputs.File()
output_file_path = gr.outputs.Text(type="file")

gr.Interface(get_file_path, inputs=input_file, outputs=output_file_path).launch()


D:\Anaconda\envs\ML\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
D:\Anaconda\envs\ML\lib\site-packages\gradio\deprecation.py:40: UserWarning: `keep_filename` parameter is deprecated, and it has no effect
  warnings.warn(value)


AttributeError: module 'gradio.outputs' has no attribute 'Text'